The ORACLE DB MUST be completely done before this section:

* Update the **username**, **password** and **cluster** information to your own MongoDB Atlas Cluster
* Create the tournaments, rounds, hands and players collections in your cluster before running queries below. 

This helps to avoid hiccups along the way.


In [ ]:
!pip install 
!pip install oracledb
!pip install line_profiler
!pip install memory_profiler

In [ ]:
%load_ext memory_profiler
%load_ext line_profiler

%reload_ext memory_profiler
%reload_ext line_profiler

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [ ]:
username = 'admin'
password = 'password'
cluster = 'cluster'
uri = f"mongodb+srv://{username}:{password}@{cluster}/?retryWrites=true&w=majority"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Successfully connected to MongoDB!")
except Exception as e:
    print(f"Connection failed: {e}")

In [ ]:
#Create analytics database and collections

db = client['PokerTournamentAnalytics']
players_collection = db['Players']
tournaments_collection = db['Tournaments']
rounds_collection = db['Rounds']
hands_collection = db['Hands']

##### Connect to the oracle database

In [ ]:
import oracledb
host = '127.0.0.1'
port = '1521'
sname = 'orclpdb1'

username = 'admin'
password = 'password'

dsn = f"(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={host})(PORT={port}))(CONNECT_DATA=(SERVICE_NAME={sname})))"

# Establish the connection
try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    cursor = connection.cursor()
    print("Connection successful!")
except oracledb.DatabaseError as e:
    print("There was an error connecting to the database:", e)

### Players Collection

In [ ]:
#PLAYERS COLLECTION
query = '''
SELECT * FROM PLAYERS
ORDER BY PID
'''
connection.execute(query)
players_data = connection.fetchall()

#***************************
query = '''
SELECT distinct p.PID, tw.tid, t.tname, tw.payout,ROW_NUMBER() OVER (PARTITION BY tw.tid ORDER BY tw.payout DESC) AS Pos
FROM players p
INNER JOIN TOURNAMENTSWINNERS tw on p.pid = tw.pid
INNER JOIN tournaments t on t.tid = tw.tid
ORDER BY p.PID
'''
connection.execute(query)
playerswin_data = connection.fetchall()

#***************************
query = '''
SELECT DISTINCT p.pid, t.tname
FROM players p
INNER JOIN handplayers hp on hp.pid = p.pid
INNER JOIN hands h on hp.handid = h.handid
INNER JOIN games g on g.gameid = h.gameid
INNER JOIN rounds r on r.roundid = g.roundid
INNER JOIN tournaments t on t.tid = r.tid
ORDER BY PID
'''
connection.execute(query)
playersPlayedIn_data = connection.fetchall()

#***************************

In [ ]:
#PUSH PLAYERS COLLECTION TO MONGO DB
def addPlayersData():
    Players = []

    for p in players_data:
        pid, pname, pAge, pcountry = p

        wins = [ {"TName": p[2],"Payout": p[3],"Position":p[4]} for p in playerswin_data if p[0] == pid]
        plays = [ pi[1] for pi in playersPlayedIn_data if pi[0] == pid]

        players_info = {
            'PID': pid,
            'Name': pname,
            'Age': pAge,
            'Country': pcountry,
            'Wins': wins,
            'PlayedIn': plays
        }
        Players.append(players_info)
    i = players_collection.insert_many(Players)

In [ ]:
addPlayersData()
players_collection..count_documents({})

### Tournaments Collection

In [ ]:
#TOURNAMENTS COLLECTION
q1 = '''
SELECT * FROM tournaments
'''
connection.execute(q1)

tournament_data = connection.fetchall()

#*****************************
q2 = '''
SELECT DISTINCT t.tid, hp.pid
FROM tournaments t
INNER JOIN rounds r on t.tid = r.tid
INNER JOIN games g on g.roundid = r.roundid
INNER JOIN hands h on h.gameid = g.gameid
INNER JOIN handplayers hp on hp.handid = h.handid
ORDER BY t.TID, hp.PID
'''
connection.execute(q2)

tournament_players_data = connection.fetchall()

#***************************
q3 = '''
--TOURNAMENT WINNERS -add to a list or set
SELECT ROW_NUMBER() OVER (PARTITION BY tid ORDER BY payout DESC) AS TPosition, tw.TID, pname, tw.PID,tw.payout
FROM TOURNAMENTSWINNERS tw
INNER JOIN players p on p.pid = tw.pid
ORDER BY tid, payout DESC
'''
connection.execute(q3)

tournament_winners_data = connection.fetchall()

In [ ]:
def createTournamentsCollection():
    Tournaments = []

    for t in tournament_data:
        tid, name, location, date, buyin = t
        type(date)

        # Convert the datetime object to a string in "YYYY-MM-DD" format
        fm_date = date.strftime('%Y-%m-%d')

        players = [ p[1] for p in tPlayers_data if p[0] == tid]
        winners = [{"Position":w[0], "Player":w[2], "Player_id":w[3], "Payout":w[4]} for w in tWinners_data if w[1] == tid]

        tournament_info = {
            'TID': tid,
            'Name': name,
            'Date': fm_date,
            'Location': location,
            'Players': players,
            'Winners': winners
        }
        Tournaments.append(tournament_info)

    i = tournaments_collection.insert_many(Tournaments)

In [ ]:
addTournamentsData()
tournaments_collection.count_documents({})

### Rounds Collection

In [ ]:
query = '''
SELECT * FROM rounds
'''
connection.execute(query)
rounds_data = connection.fetchall()

#***************************

query = '''
SELECT gameid, gamenum, roundid as game_roundid FROM games
'''
connection.execute(query)
games_data = connection.fetchall()

#***************************

query = '''
SELECT handid, gameid FROM hands
'''
connection.execute(query)
hands_data = connection.fetchall()

In [ ]:
def addRoundsData():
    rounds = []

    for round in rounds_data:
        roundid, tid, roundnum = round

        games = []
        for game in games_data:
            gameid, gamenum, game_roundid = game

            if game_roundid == roundid:
                game =  {
                    'game_id': gameid,
                    'game number': gamenum,
                    'hands': [ {"hand_id": h[0]} for h in hands_data if h[1] == gameid ]
                }
                games.append(game)

        round_info = {
            'Tournament_id': tid,
            'Round_id': roundid,
            'Round Number': roundnum,
            'Games': games
        }
        rounds.append(round_info)

        games = []
    n = rounds_collection.insert_many(rounds)

In [ ]:
data = fetch_rounds_games_hands(connection)
rounds_data = data["rounds"]
games_data  = data["games"]
hands_data  = data["hands"]
count = add_rounds_data(rounds_data, games_data, hands_data, rounds_collection)
print(f"Inserted {count} rounds into MongoDB.")

In [ ]:
def addRoundsData():
    rounds = []

    for round in rounds_data:
        roundid, tid, roundnum = round

        games = []
        for game in games_data:
            gameid, gamenum, game_roundid = game

            if game_roundid == roundid:
                game =  {
                    'game_id': gameid,
                    'game number': gamenum,
                    'hands': [ {"hand_id": h[0]} for h in hands_data if h[1] == gameid ]
                }
                games.append(game)

        round_info = {
            'Tournament_id': tid,
            'Round_id': roundid,
            'Round Number': roundnum,
            'Games': games
        }
        rounds.append(round_info)

        games = []
    n = rounds_collection.insert_many(rounds)

In [ ]:
addRoundsData()
rounds_collection.count_documents({})

### Hands Collection

In [ ]:
#HANDS COLLECTION
q1 = '''
SELECT handid, handnum, dealer, fc1, fc2, fc3, tc1, rc1 from hands hs
INNER JOIN board b on b.boardid = hs.boardid
'''
connection.execute(q1)
hands_board_data = connection.fetchall()

#*****************************

q2 = '''
SELECT hs.handid, p.pid, p.pname, card1, card2, act, chipamt, pos 
FROM hole h
INNER JOIN handplayers hp on hp.hpid = h.hpid
INNER JOIN hands hs on hs.handid = hp.handid
INNER JOIN players p on p.pid = hp.pid
'''

connection.execute(q2)
hole_data = connection.fetchall()
#*****************************

q3 = '''
SELECT handid, p.pid, p.pname, stage, handrank, act, chipamt
FROM cards c
INNER JOIN handplayers hp on hp.hpid = connection.hpid
INNER JOIN players p on p.pid = hp.pid
'''

connection.execute(q3)
cards_data = connection.fetchall()
#*****************************

q4 = '''
SELECT h.handid, p.pid, pname, pot, connection.handrank, pos
FROM handwinners hw
INNER JOIN handplayers hp on hp.hpid = hw.hpid
INNER JOIN hands h on h.handid = hp.handid
INNER JOIN cards c on connection.hpid = hw.hpid
INNER JOIN players p on p.pid = hp.pid
WHERE STAGE = 'RIVER'
'''

connection.execute(q4)
handwinners_data = connection.fetchall()
#*****************************

q5 = '''
SELECT handid, p.pid, p.pname, pos as position 
FROM handplayers hp
INNER JOIN players p on p.pid = hp.pid
'''
connection.execute(q5)
handplayers_data = connection.fetchall()
#***************************

query = '''
SELECT h.handid, p.pname, p.pid
FROM hands h
INNER JOIN handplayers hp on hp.handid = h.handid
INNER JOIN continuing c on connection.hpid = hp.hpid
inner JOIN players p on p.pid = hp.pid
'''

connection.execute(query)
continuing_players_data = connection.fetchall()

#***************************

query = '''
SELECT h.handid, h.handnum, p.pname, p.pid
FROM hands h
INNER JOIN handplayers hp on hp.handid = h.handid
INNER JOIN eliminated e on e.hpid = hp.hpid
inner JOIN players p on p.pid = hp.pid
'''

connection.execute(query)
eliminated_players_data = connection.fetchall()

In [ ]:
def addHandsData():
    Hands = []

    for hand in hands_board_data:
        handid, handnum, dealer, FC1, FC2, FC3, TC1, RC1 = hand

        board = {"Flop 1": FC1, "Flop 2": FC2, "Flop 3": FC3, "Turn": TC1, "River" : RC1}

        holes = [ {"Player_id": hole[1], "Player":hole[2], "Cards": [ hole[3], hole[4] ], "Action": hole[5], "Chips": hole[6]}  for hole in hole_data if hole[0] == handid ]

        cards = [ {"Player_id": card[1], "Player":card[2], "Stage": card[3], "HandRank": card[4], "Action": card[5], "Chips": card[6]} for card in cards_data if card[0] == handid]

        players = [ {"Player_id": p[1], "Player":p[2], "Position": p[3]} for p in handplayers_data if p[0] == handid ]
        
        continued = [ {"Player_id": cp[2], "Player": cp[1]} for cp in continuing_players_data if cp[0] == handid]
            
        eliminated = [ {"Player_id": ep[3], "Player": ep[2], "eliminated_in_hand": ep[1]} for ep in eliminated_players_data if ep[0] == handid ]

        winners = []
        for hw in handwinners_data:
            if hw[0] == handid:
                if winners and winners[0] == hw[1]: #avoid duplicate entries - there are usually two winners or less
                    continue
                winners = [ {"Player_id":hw[1], "Player":hw[2], "Pot":hw[3], "HandRank":hw[4], "Position":hw[5]}]
   

        hand_info = {
            'hand_id': handid,
            'board': board,
            'hole': holes,
            'cards': cards,
            'dealer': dealer,
            'winners': winners,
            'players': players,
            'continued': continued,
            'eliminated': eliminated
        }
        Hands.append(hand_info)
        players = []
        cards = []
        holes = []
        continued = []
        eliminated = []
    x = hands_collection.insert_many(Hands)

In [ ]:
addHandsData()
hands_collection.count_documents({})

In [ ]:
#close oracle db and mongodb connections
connection.close()
client.close()